# NCAA Analysis

Summary: This notebook creates a weighted ranking of the NCAA Division 1 Men's Basketball Teams using data available on CBS Sportsline. Total games played are tracked in a full matrix. The matrix also tracks when teams play each other. A weighted vector is used to weight the results. Games won are worth one point, games lost are worth 0.5 points. Games won on the road and blowouts (greater than 20 points) earn extra. Winning games are also scored higher when the games are won later in the season. 

In [112]:
#Import Dependencies
import pandas as pd
import numpy as np
import math
import requests
from bs4 import BeautifulSoup as soup
import re
import collections
import pickle

In [113]:
#Global Variables for Weighting
AWAY = 0.1
BLOWOUT = 0.1

In [114]:
def Match(pattern, string):
    if re.match(pattern, string):
        return True
    else:
        return False 

# Create a DataFrame Which Contains All Teams Available on CBS Sportsline

In [116]:
def create_team_df():
    '''Go to CBS Sports and pull data from all NCAA Mens Basketball using the pull down available in the first
    page (url). The function creates a dataframe for the link to each teams results, the team name, the teams 
    shortname, and the teams full name'''
    url = 'http://www.cbssports.com/collegebasketball/teams/schedule/ALBANY/albany-great-danes'
    schedule = requests.get(url)
    content = schedule.content
    html_txt = soup(content, 'lxml')
    divs = html_txt.find_all('div' , { "class" : 'fRight' })
    all_teams = []
    for div in divs:
        div = str(div)
        if 'pulldown' in div:
            teams = div.split('<option value="')
            for team in teams:
                team = team.split('">')
                team[1] = team[1].replace('</option>', '').replace('</select></form></div>', '').replace('A&amp;M ', '')
                if '/collegebasketball' in team[0]:
                    link = team[0].split('/')
                    team.append(link[4])
                    team.append(link[5])
                    all_teams.append(team)
    columns = ['link', 'team', 'shortname', 'longname']
    return pd.DataFrame(all_teams, columns=columns)
teams = create_team_df()
teams.head()

,link,team,shortname,longname
0,/collegebasketball/teams/schedule/ABIL/abilene...,Abilene Christian Wildcats,ABIL,abilene-christian-wildcats
1,/collegebasketball/teams/schedule/AF/air-force...,Air Force Falcons,AF,air-force-falcons
2,/collegebasketball/teams/schedule/AKRON/akron-...,Akron Zips,AKRON,akron-zips
3,/collegebasketball/teams/schedule/ALAM/alabama...,Alabama Bulldogs,ALAM,alabama-am-bulldogs
4,/collegebasketball/teams/schedule/BAMA/alabama...,Alabama Crimson Tide,BAMA,alabama-crimson-tide


# Compile and Score Results from All Games

In [117]:
def schedule_weights(game):
    '''Weight winning games based on when games are played.'''
    if int(game['date']) < 20161220:
        rank = -0.1
    elif int(game['date']) >= 20161220 and int(game['date']) < 20170215:
        rank = -0.05
    else:
        rank = 0
    return rank

def rank_result(team_score, opponent_score, home_game):
    '''Weight game results by home game, whether the game was a blowout, and when the game was played'''
    rank = 0
    if int(team_score) > int(opponent_score):
        rank += 1
        if int(home_game) == 0:
            rank += AWAY
        if int(team_score) - int(opponent_score) > 20:
            rank += BLOWOUT
        rank += schedule_weights(game)
    elif int(team_score) == int(opponent_score):
        rank = 0
    else:
        rank +=(-0.5)
    return rank

In [6]:
def split_opponent(game, home_team):
    game = game.split('@')
    if home_team == game[1]:
        return game[0], 1
    else:
        return game[1], 0
    
def split_result(result): 
    result = result.strip()
    result = result.split(' ')
    score = result[2].replace('</a>', '')
    score = score.split('-')
    played = -1
    return played, score[0], score[1] 

def get_team_results(team):
    '''Go to CBS Sportsline and scrape the results of each game played during the season for team passed into 
    the function. Return a dictionary with the results of each game.'''
    url = 'http://www.cbssports.com/collegebasketball/teams/schedule/' + team
    schedule = requests.get(url)
    content = schedule.content
    html_txt = soup(content, 'lxml')
    links = html_txt.find_all('a')
    results = {}
    counter = 0
    for link in links:
        if '/collegebasketball/gametracker/recap/' in str(link):
            result = str(link).split('_')
            counter += 1
            game = team + '_' + str(counter)
            date = result[1]
            rest = result[2].split('">')
            opponent, home_game = split_opponent(rest[0], team)
            played, team_score, opponent_score = split_result(rest[1])
            results[game] = {'date' : date, 'opponent' : opponent, 'home_game' : home_game, 'result' : played,
                            'team_score' : team_score, 'opponent_score' : opponent_score, 'team' : team}
    return results






# Initiate the Data Collection.

In [57]:
#Create a columns variable to use in the Results Grid DataFrame
columns = ['team', 'home_game', 'opponent_score', 'team_score', 'result', 'date', 'opponent']
results_grid = pd.DataFrame(columns=columns)
status = 0 #Used for tracking the status of the scraping (350+ teams)
#Use Team Data Frame to call the get_team_results() function
for team in teams['shortname']:
    game_count = 2
    results = get_team_results(team)
    for result in results:
        results_grid.loc[result] = pd.Series(results[result])
        game_count += 1
    team_rank = 'team' + '_rank'
    #Create a row in the DataFrame to store the number of games played.
    ranking = {'team': team, 'opponent' : team, 'result' : game_count}
    results_grid.loc[team] = pd.Series(ranking) 
    status += 1
    if status % 10 == 0:
        print 'On Team Number:', status
results_grid = results_grid.fillna(0) #Remove Null Values
results_grid.to_pickle('results_grid.pkl') #Save Results to a pickeled file
results_grid.head()

On Team Number: 10
On Team Number: 20
On Team Number: 30
On Team Number: 40
On Team Number: 50
On Team Number: 60
On Team Number: 70
On Team Number: 80
On Team Number: 90
On Team Number: 100
On Team Number: 110
On Team Number: 120
On Team Number: 130
On Team Number: 140
On Team Number: 150
On Team Number: 160
On Team Number: 170
On Team Number: 180
On Team Number: 190
On Team Number: 200
On Team Number: 210
On Team Number: 220
On Team Number: 230
On Team Number: 240
On Team Number: 250
On Team Number: 260
On Team Number: 270
On Team Number: 280
On Team Number: 290
On Team Number: 300
On Team Number: 310
On Team Number: 320
On Team Number: 330
On Team Number: 340
On Team Number: 350


,team,home_game,opponent_score,team_score,result,date,opponent
ABIL_28,ABIL,1.0,89,75,-1.0,20170301,UIW
ABIL_29,ABIL,0.0,81,72,-1.0,20170304,HOUBP
ABIL_24,ABIL,1.0,72,76,-1.0,20170211,NWST
ABIL_25,ABIL,0.0,78,82,-1.0,20170216,MCNSE
ABIL_26,ABIL,1.0,56,44,-1.0,20170222,TXAMCC


# Create a Results Vector to Score Each Game

In [131]:
results_grid['Value'] = results_grid.apply(lambda row: rank_result(row['team_score'], row['opponent_score'], row['home_game']), axis=1)
results_cat = results_grid.filter(['team', 'result', 'opponent', 'Value']) #Create Smaller DataFrame for Results Matrix
results_vec = results_cat.groupby(['team'])['Value'].sum() #Sum all games by team to store in the results vector
print results_vec.head(), results_vec.tail()

team
ABIL       4.2
AF        -0.2
AKRON     20.7
ALAM     -11.6
ALBANY    13.7
Name: Value, dtype: float64 team
WVU       20.1
WYO        9.7
XAVIER    13.6
YALE      11.8
YOUNG      1.8
Name: Value, dtype: float64


# Create a Matrix for all Games Played

In [119]:
#Pivot smaller DF into a Results Matrix
results_values = results_cat.pivot_table(index='team', columns='opponent', values='result') 
results_values = results_values.fillna(0) #Fill in NA Values to 0 (Team Not Played)
results_values.to_pickle('results_values.pkl') #Save Results to a Pickled Dataset
#Used to filter games of teams played which were not in the Teams DataFrame
in_list = results_values.index.values.tolist() 
top_team_results = results_values.filter(in_list)

In [120]:
print top_team_results, '\n', results_vector

opponent  ABIL    AF  AKRON  ALAM  ALBANY  ALCORN  ALST  AMER  APPST  ARIZ  \
team                                                                         
ABIL      31.0   0.0    0.0   0.0     0.0     0.0   0.0   0.0    0.0   0.0   
AF         0.0  34.0   -1.0   0.0     0.0     0.0   0.0   0.0    0.0   0.0   
AKRON      0.0  -1.0   36.0   0.0     0.0     0.0   0.0  -1.0    0.0   0.0   
ALAM       0.0   0.0    0.0  31.0     0.0    -1.0  -1.0   0.0    0.0   0.0   
ALBANY     0.0   0.0    0.0   0.0    36.0     0.0   0.0   0.0    0.0   0.0   
ALCORN     0.0   0.0    0.0  -1.0     0.0    34.0  -1.0   0.0    0.0   0.0   
ALST       0.0   0.0    0.0  -1.0     0.0    -1.0  33.0   0.0    0.0   0.0   
AMER       0.0   0.0   -1.0   0.0     0.0     0.0   0.0  32.0    0.0   0.0   
APPST      0.0   0.0    0.0   0.0     0.0     0.0   0.0   0.0   32.0   0.0   
ARIZ       0.0   0.0    0.0   0.0     0.0     0.0   0.0   0.0    0.0  36.0   
ARIZST     0.0   0.0    0.0   0.0     0.0     0.0   0.0   0.0   

#Testing Only with smaller list of Teams
columns = ['team', 'home_game', 'opponent_score', 'team_score', 'result', 'date', 'opponent']
results_grid = pd.DataFrame(columns=columns)
status = 0
results_vector = collections.OrderedDict()
for team in team_list:
    rank_sum = 0
    game_count = 2
    results = get_team_results(team)
    for result in results:
        rank_sum += rank_result(results[result])
        results_grid.loc[result] = pd.Series(results[result])
        game_count += 1
    team_rank = 'team' + '_rank'
    ranking = {'team': team, 'opponent' : team, 'result' : game_count}
    results_grid.loc[team] = pd.Series(ranking) 
    results_vector[team] = rank_sum
    status += 1
    if status % 10 == 0:
        print 'On Team Number:', status
#results = get_team_results('PSU')
results_grid.head()

 

# Use Linear Algebra to Rank Teams

In [132]:
ratings = pd.Series(np.linalg.solve(top_team_results, results_vec), index=top_team_results.index) 
rankings = ratings.sort_values(ascending=False)
rankings = pd.DataFrame(rankings, columns=['rating'])
#Reindex the values
rankings['team'] = rankings.index #Copy index (team values) to new column
index = []
for value in range(len(rankings)):
    index.append(value + 1)
rankings.index = index #Set Integer Index
    

In [125]:
#Use the Teams DataFrame to Merge Team Data with the Rankings
rankings2 = pd.merge(rankings, teams, left_on='team', right_on='shortname')
rankings2.head(60)

,rating,team_x,link,team_y,shortname,longname
0,1.366218,UK,/collegebasketball/teams/schedule/UK/kentucky-...,Kentucky Wildcats,UK,kentucky-wildcats
1,1.322544,UNC,/collegebasketball/teams/schedule/UNC/north-ca...,North Carolina Tar Heels,UNC,north-carolina-tar-heels
2,1.315738,LVILLE,/collegebasketball/teams/schedule/LVILLE/louis...,Louisville Cardinals,LVILLE,louisville-cardinals
3,1.314211,GONZAG,/collegebasketball/teams/schedule/GONZAG/gonza...,Gonzaga Bulldogs,GONZAG,gonzaga-bulldogs
4,1.284915,FSU,/collegebasketball/teams/schedule/FSU/florida-...,Florida State Seminoles,FSU,florida-state-seminoles
5,1.281272,NOVA,/collegebasketball/teams/schedule/NOVA/villano...,Villanova Wildcats,NOVA,villanova-wildcats
6,1.270651,ARIZ,/collegebasketball/teams/schedule/ARIZ/arizona...,Arizona Wildcats,ARIZ,arizona-wildcats
7,1.263184,FLA,/collegebasketball/teams/schedule/FLA/florida-...,Florida Gators,FLA,florida-gators
8,1.262605,PURDUE,/collegebasketball/teams/schedule/PURDUE/purdu...,Purdue Boilermakers,PURDUE,purdue-boilermakers
9,1.254613,DUKE,/collegebasketball/teams/schedule/DUKE/duke-bl...,Duke Blue Devils,DUKE,duke-blue-devils


In [123]:
rankings2.loc[rankings2['team_x'] == 'GONZAG']

,rating,team_x,link,team_y,shortname,longname
3,1.314211,GONZAG,/collegebasketball/teams/schedule/GONZAG/gonza...,Gonzaga Bulldogs,GONZAG,gonzaga-bulldogs


In [126]:
print rankings2.loc[rankings2['team_x'] == 'CREIGH'], rankings2.loc[rankings2['team_x'] == 'RI'], 

      rating  team_x                                               link  \
53  0.973928  CREIGH  /collegebasketball/teams/schedule/CREIGH/creig...   

                team_y shortname            longname  
53  Creighton Bluejays    CREIGH  creighton-bluejays         rating team_x                                               link  \
28  1.097617     RI  /collegebasketball/teams/schedule/RI/rhode-isl...   

               team_y shortname           longname  
28  Rhode Island Rams        RI  rhode-island-rams  


In [83]:
results = results.sort_values(ascending=False)


In [133]:
#Use Results Vector to Cross Analyze the Rankings
df_results = pd.DataFrame(results_vec)
df_results.columns = ['rank_result']
df_results['team'] = df_results.index
index = []
for value in range(len(df_results)):
    index.append(value + 1)
df_results = df_results.sort_values(by='rank_result', ascending=False)
df_results.index = index
df_results

,rank_result,team
1,31.5,GONZAG
2,28.7,NOVA
3,27.2,WICHST
4,26.9,SMU
5,26.8,ARIZ
6,26.7,MTSU
7,25.9,UK
8,25.8,UCLA
9,25.5,OREG
10,25.4,CINCY


In [105]:
print df_results.loc[df_results['team'] == 'CREIGH'], df_results.loc[df_results['team'] == 'RI'], 

    rank_result    team
35         19.4  CREIGH     rank_result team
43         18.5   RI
